In [1]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import upath

# import npc_lims

In [2]:
results_path=upath.UPath("s3://aind-scratch-data/dynamic-routing/decoding/results/validate_v260_20250401_030903/")

In [3]:
session_table_path=r"D:\session_table_v0.260.csv"
session_table=pl.read_csv(session_table_path)

In [22]:
dr_session_list=(
    session_table.filter(
    pl.col('project').eq('DynamicRouting'),
    pl.col('is_production').eq(True),
    pl.col('is_good_behavior').eq(True),
    )['session_id'].to_list()
    )
dr_session_list

['626791_2022-08-15',
 '636766_2023-01-23',
 '636766_2023-01-24',
 '644864_2023-02-02',
 '644866_2023-02-08',
 '644866_2023-02-09',
 '644866_2023-02-10',
 '649943_2023-02-13',
 '649943_2023-02-14',
 '649943_2023-02-15',
 '644867_2023-02-20',
 '644867_2023-02-22',
 '644867_2023-02-23',
 '660023_2023-08-09',
 '666986_2023-08-14',
 '666986_2023-08-15',
 '666986_2023-08-16',
 '666986_2023-08-17',
 '662892_2023-08-22',
 '662892_2023-08-23',
 '662892_2023-08-24',
 '668755_2023-08-29',
 '668755_2023-08-31',
 '667252_2023-09-26',
 '667252_2023-09-27',
 '667252_2023-09-28',
 '674562_2023-10-02',
 '674562_2023-10-03',
 '674562_2023-10-04',
 '674562_2023-10-05',
 '681532_2023-10-18',
 '686740_2023-10-24',
 '686740_2023-10-25',
 '664851_2023-11-13',
 '664851_2023-11-15',
 '664851_2023-11-16',
 '690706_2023-11-28',
 '690706_2023-11-30',
 '686176_2023-12-04',
 '686176_2023-12-07',
 '702131_2024-02-26',
 '703880_2024-04-15',
 '706401_2024-04-23',
 '706401_2024-04-24',
 '708016_2024-04-29',
 '708016_2

In [23]:
#define grouping columns
grouping_cols = {
    'session_id',
    'structure',
    'electrode_group_names',
    'min_n_units',
    'unit_criteria',
}

#toggle combine_multi_probe_rec
combine_multi_probe_rec = True

if combine_multi_probe_rec:
    combine_multi_probe_expr = pl.col('electrode_group_names').list.len().gt(1) | pl.col('is_sole_recording').eq(True)
else:
    combine_multi_probe_expr = pl.col('electrode_group_names').list.len().eq(1) | pl.col('is_sole_recording').eq(True)
 
# (
#     pl.scan_parquet(results_path)
#     .with_columns(
#         pl.col('electrode_group_names').list.n_unique().eq(1).over(grouping_cols - {'electrode_group_names'}).alias('is_sole_recording'),
#     )
#     .filter(combine_multi_probe_expr)
# )

In [ ]:
#templeton session list=

#good behavior list=

In [ ]:
df = (
    pl.scan_parquet(results_path)
    .with_columns(
        pl.col('electrode_group_names').list.n_unique().eq(1).over(grouping_cols - {'electrode_group_names'}).alias('is_sole_recording'),
    )
    .filter(
        combine_multi_probe_expr,
        pl.col('is_all_trials').not_(),
        pl.col('session_id').is_in(dr_session_list),
        pl.col('session_id').n_unique().ge(3).over('structure', 'min_n_units', 'unit_criteria'),
    )
    # get the means for each recording over their shifts:
    .group_by(grouping_cols | {'repeat_idx'})
    .agg(
        pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0)).mean().alias('mean_true'),
        pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0).not_()).median().alias('median_null'),
        pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0)).mean().sub(
            pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0).not_()).median()
        ).alias('mean_diff')
    )
    # get the means over repeats:
    .group_by(grouping_cols)
    .agg(
        pl.col('mean_true').mean(),
        pl.col('median_null').mean(),
        pl.col('mean_diff').mean()
    )   
    # get the means over sessions:
    .group_by('structure', 'min_n_units', 'unit_criteria')
    .agg(
        pl.col('mean_true').mean(),
        pl.col('median_null').mean(),
        pl.col('mean_diff').mean()
    )
    .sort(pl.col('mean_diff').mean().over('structure'), descending=True)
    .collect(streaming=True)
)

In [25]:
df

structure,min_n_units,unit_criteria,mean_true,median_null,mean_diff
str,u8,cat,f64,f64,f64
"""SCiw""",20,"""medium""",0.799538,0.650441,0.149097
"""FF""",20,"""medium""",0.733724,0.599349,0.134375
"""SCm""",20,"""medium""",0.766981,0.637529,0.129452
"""SCig""",20,"""medium""",0.733073,0.618779,0.114293
"""SNr""",20,"""medium""",0.718271,0.609213,0.109058
…,…,…,…,…,…
"""TTd""",20,"""medium""",0.550042,0.549854,0.000188
"""RN""",20,"""medium""",0.540709,0.547324,-0.006615
"""ACB""",20,"""medium""",0.55382,0.562772,-0.008952


In [ ]:
df_session_wise = (
    pl.scan_parquet(results_path)
    .with_columns(
        pl.col('electrode_group_names').list.n_unique().eq(1).over(grouping_cols - {'electrode_group_names'}).alias('is_sole_recording'),
    )
    .filter(
        combine_multi_probe_expr,
        pl.col('is_all_trials').not_(),
        pl.col('session_id').n_unique().ge(3).over('structure', 'min_n_units', 'unit_criteria'),
    )
    # get the means for each recording over their shifts:
    .group_by(grouping_cols | {'repeat_idx'})
    .agg(
        pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0)).mean().alias('mean_true'),
        pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0).not_()).median().alias('median_null'),
        pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0)).mean().sub(
            pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0).not_()).median()
        ).alias('mean_diff')
    )
    # get the means over repeats:
    .group_by(grouping_cols)
    .agg(
        pl.col('mean_true').mean(),
        pl.col('median_null').mean(),
        pl.col('mean_diff').mean()
    )   
    .sort(pl.col('session_id'), descending=True)
    .collect(streaming=True)
)

In [27]:
df_session_wise

unit_criteria,structure,session_id,electrode_group_names,min_n_units,mean_true,median_null,mean_diff
cat,str,str,list[str],u8,f64,f64,f64
"""medium""","""PoT""","""742903_2024-10-24""","[""probeE""]",20,0.524042,0.520824,0.003218
"""medium""","""SSp""","""742903_2024-10-24""","[""probeA""]",20,0.546465,0.545699,0.000766
"""medium""","""CP""","""742903_2024-10-24""","[""probeB""]",20,0.593697,0.565113,0.028584
"""medium""","""CP""","""742903_2024-10-24""","[""probeA""]",20,0.561859,0.545263,0.016596
"""medium""","""CP""","""742903_2024-10-24""","[""probeF""]",20,0.585473,0.548067,0.037406
…,…,…,…,…,…,…,…
"""medium""","""DG""","""626791_2022-08-15""","[""probeA"", ""probeB""]",20,0.634306,0.601254,0.033052
"""medium""","""CA1""","""626791_2022-08-15""","[""probeA"", ""probeC"", ""probeB""]",20,0.589964,0.576814,0.01315
"""medium""","""LP""","""626791_2022-08-15""","[""probeA""]",20,0.629038,0.575818,0.05322


In [ ]:
# pl.scan_parquet(results_path).columns

In [ ]:
# all_results=pl.scan_parquet(results_path)
# all_results

In [5]:
# Try with pandas
all_results_pd=pd.read_parquet(results_path)

In [ ]:
all_results_pd.to_parquet(r"D:\decoding_results_from_CO\new_decoder_run_2025-03-31\all_results_pd.parquet")

In [9]:
all_results_pd.query('is_all_trials')

,balanced_accuracy_test,shift_idx,repeat_idx,predict_proba,unit_ids,is_all_trials,session_id,structure,electrode_group_names,min_n_units,n_units,unit_criteria
89,0.868412,NaN,0,"[0.7378876996227072, 0.5555895036877662, 0.980...","[667252_2023-09-26_C-252, 667252_2023-09-26_C-...",True,667252_2023-09-26,SCiw,[probeC],20,20,medium
179,0.854361,NaN,1,"[0.7558875360757467, 0.659520885235395, 0.9458...","[667252_2023-09-26_C-252, 667252_2023-09-26_C-...",True,667252_2023-09-26,SCiw,[probeC],20,20,medium
269,0.866593,NaN,2,"[0.7289819469993282, 0.4190290980883542, 0.933...","[667252_2023-09-26_C-252, 667252_2023-09-26_C-...",True,667252_2023-09-26,SCiw,[probeC],20,20,medium
359,0.841375,NaN,3,"[0.6970223241270189, 0.4309393137705571, 0.906...","[667252_2023-09-26_C-252, 667252_2023-09-26_C-...",True,667252_2023-09-26,SCiw,[probeC],20,20,medium
449,0.862597,NaN,4,"[0.8744690709397488, 0.3638774016648089, 0.898...","[667252_2023-09-26_C-252, 667252_2023-09-26_C-...",True,667252_2023-09-26,SCiw,[probeC],20,20,medium
...,...,...,...,...,...,...,...,...,...,...,...,...
5714079,0.566007,NaN,20,"[0.7583472690463605, 0.8408958126042063, 0.796...","[681532_2023-10-18_C-49, 681532_2023-10-18_B-4...",True,681532_2023-10-18,LP,"[probeB, probeC]",20,20,medium
5714159,0.623463,NaN,21,"[0.6828661821077902, 0.7145846841409936, 0.744...","[681532_2023-10-18_C-51, 681532_2023-10-18_C-4...",True,681532_2023-10-18,LP,"[probeB, probeC]",20,20,medium
5714239,0.614357,NaN,22,"[0.5569938287594297, 0.6635757200487805, 0.838...","[681532_2023-10-18_B-132, 681532_2023-10-18_B-...",True,681532_2023-10-18,LP,"[probeB, probeC]",20,20,medium
5714319,0.624679,NaN,23,"[0.8336814560583709, 0.8306010505596675, 0.796...","[681532_2023-10-18_B-132, 681532_2023-10-18_B-...",True,681532_2023-10-18,LP,"[probeB, probeC]",20,20,medium


In [20]:
# all_results_pd['electrode_group_names'].unique()

In [21]:
session_table.columns

['session_id',
 'is_production',
 'project',
 'date',
 'time',
 'subject',
 'subject_age_days',
 'subject_sex',
 'subject_genotype',
 'implant',
 'rig',
 'experimenters',
 'notes',
 'issues',
 'allen_path',
 'cloud_path',
 'task_version',
 'ephys_day',
 'behavior_day',
 'epochs',
 'is_ephys',
 'is_deep_insertions',
 'is_sync',
 'is_video',
 'is_templeton',
 'is_annotated',
 'is_hab',
 'is_task',
 'is_invalid_times',
 'is_naive',
 'is_context_naive',
 'is_late_autorewards',
 'is_spontaneous',
 'is_spontaneous_rewards',
 'is_rf_mapping',
 'is_optotagging',
 'is_optotagging_control',
 'is_opto_perturbation',
 'is_injection_perturbation',
 'is_opto_control',
 'is_injection_control',
 'probe_letters_available',
 'probe_letters_to_skip',
 'probe_letters_annotated',
 'deep_probe_letters_to_skip',
 'areas_perturbed',
 'areas_recorded',
 'task_duration',
 'mean_intra_modal_dprime_vis',
 'mean_intra_modal_dprime_aud',
 'n_passing_blocks',
 'cross_modal_dprime_vis_blocks',
 'cross_modal_dprime_au

In [ ]:
#can I recreate my previous tables with the new data format?


#session-area wise decoding accuracy & predict_proba



In [ ]:
#validating new decoder results:

#1) compare overall null corrected accuracy for each area (paired barplot)
#2) compare session-wise true accuracy, null accuracy, and accuracy above null for every area in every recording - correlation?
#3) compare trial-trial correlation within area & session

#4) control - shuffle session id? or shuffle area id?